<div dir="ltr" style="text-align: left;" trbidi="on">
<h2 style="text-align: left;">
100 Days of ML Day 68</h2>
<h3 style="text-align: left;">
HackerEarth Airplane Accident Severity Challenge Part 5</h3>
Tried using an ensemble of Deep Neural Networks to classify severity of the Airplane Accident<br />
<br />
Code: <a href="https://github.com/hithesh111/Hith100/blob/master/100Days/day068.ipynb">https://github.com/hithesh111/Hith100/blob/master/100Days/day068.ipynb</a><br />
<br />
Competition: <a href="https://www.hackerearth.com/challenges/competitive/airplane-accident-severity-hackerearth-machine-learning-challenge/">https://www.hackerearth.com/challenges/competitive/airplane-accident-severity-hackerearth-machine-learning-challenge/</a><br />
<br />
Leaderboard: <a href="https://www.hackerearth.com/challenges/competitive/airplane-accident-severity-hackerearth-machine-learning-challenge/leaderboard/how-severe-can-an-airplane-accident-be-03e7a3f1/">https://www.hackerearth.com/challenges/competitive/airplane-accident-severity-hackerearth-machine-learning-challenge/leaderboard/how-severe-can-an-airplane-accident-be-03e7a3f1/</a></div>


In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras import regularizers

Using TensorFlow backend.


In [2]:
df = pd.read_csv("/home/hithesh/Downloads/Airplane Data/train.csv")
df.head()

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054


In [3]:
def normalize(df,colname):
    df[colname]=(df[colname]-df[colname].mean())/df[colname].std()
    
def standardize(df,colname):
    df[colname]=(df[colname]-df[colname].min())/(df[colname].max()-df[colname].min())

def outliercap(df,colname,factor):
    upper_lim = df[colname].mean () + df[colname].std () * factor
    lower_lim = df[colname].mean () - df[colname].std () * factor
    df.loc[(df[colname] > upper_lim),colname] = upper_lim
    df.loc[(df[colname] < lower_lim),colname] = lower_lim
    
def inv_transform(M):
    N=[]
    for i in range(M.shape[0]):
        if(max(M[i])==M[i][0]):
            N.append('Minor_Damage_And_Injuries')
        if(max(M[i])==M[i][1]):
            N.append('Significant_Damage_And_Fatalities')
        if(max(M[i])==M[i][2]):
            N.append('Significant_Damage_And_Serious_Injuries')
        if(max(M[i])==M[i][3]):
            N.append('Highly_Fatal_And_Damaging')
    return N

In [4]:
for i in [1,4,5,6,8,10]:
    normalize(df,df.columns[i])
for i in [2,3,7,9]:
    standardize(df,df.columns[i])
df['Severity'] = df['Severity'].apply(lambda x: 0 if x=='Minor_Damage_And_Injuries' else (1 if x=='Significant_Damage_And_Fatalities' else(2 if x=='Significant_Damage_And_Serious_Injuries' else (3 if x=='Highly_Fatal_And_Damaging' else -1)) ))

In [5]:
factor = 3
outliercap(df,'Safety_Score',factor)
outliercap(df,'Max_Elevation',factor)
outliercap(df,'Control_Metric',factor)
outliercap(df,'Turbulence_In_gforces',factor)

In [6]:
pred=[]
for i in range(15):
    labels = df['Severity'].unique()
    y = df['Severity']
    r=np.random.randint(1,11)
    s=np.random.randint(1,11)
    t=np.random.randint(1,11)
    u=np.random.randint(1,11)
    while(s==r or s==t or t==r):
        s=np.random.randint(1,11)
        t=np.random.randint(1,11)
    features = [df.columns[i] for i in range(1,11) if i not in [r,s,t]]
    print(features)
    X = df[features]
    l1 = tf.keras.layers.Dense(units=256,input_shape=[len(features)],activation = tf.nn.relu,kernel_regularizer=regularizers.l2(0))
    l2 = tf.keras.layers.Dense(units=256,activation = tf.nn.relu,kernel_regularizer=regularizers.l2(0))
    l3 = tf.keras.layers.Dense(units=128,activation = tf.nn.relu,kernel_regularizer=regularizers.l2(0))
    l4 = tf.keras.layers.Dense(units=4,activation = tf.nn.softmax)
    model = tf.keras.Sequential([l1,l2,l3,l4])
    model.compile(optimizer = 'adam',loss='sparse_categorical_crossentropy',metrics = ['accuracy'])
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=36)
    model.fit(X_train, y_train, epochs=200,verbose=False)
    _,accuracy=model.evaluate(X_valid,y_valid)
    print(accuracy)
    if(accuracy>0.9):
        print("Y")
        model.fit(X,y,epochs=200,verbose=False)
        df2 = pd.read_csv('/home/hithesh/Downloads/Airplane Data/test.csv')
        acc_id = df2['Accident_ID']
        for i in [0,3,4,5,7,9]:
            normalize(df2,df2.columns[i])
        for i in [1,2,6,8]:
            standardize(df2,df2.columns[i])
        X = df2[features]
        pred.append(model.predict(X))

np.array(pred).shape



['Days_Since_Inspection', 'Total_Safety_Complaints', 'Turbulence_In_gforces', 'Accident_Type_Code', 'Max_Elevation', 'Violations', 'Adverse_Weather_Metric']
2000/2000 [==============================] - 0s 63us/sample - loss: 4.6923 - accuracy: 0.4575
0.4575
['Days_Since_Inspection', 'Control_Metric', 'Turbulence_In_gforces', 'Accident_Type_Code', 'Max_Elevation', 'Violations', 'Adverse_Weather_Metric']
2000/2000 [==============================] - 0s 62us/sample - loss: 4.1476 - accuracy: 0.5385
0.5385
['Safety_Score', 'Total_Safety_Complaints', 'Control_Metric', 'Turbulence_In_gforces', 'Cabin_Temperature', 'Max_Elevation', 'Violations']
2000/2000 [==============================] - 0s 70us/sample - loss: 5.2316 - accuracy: 0.4855
0.4855
['Safety_Score', 'Total_Safety_Complaints', 'Turbulence_In_gforces', 'Accident_Type_Code', 'Max_Elevation', 'Violations', 'Adverse_Weather_Metric']
2000/2000 [==============================] - 0s 69us/sample - loss: 4.6342 - accuracy: 0.5445
0.5445
['Sa

(6, 2500, 4)

In [7]:
pred = np.array(pred)
print(pred.sum(axis=0))
df2['Severity'] = pd.Series(inv_transform(pred.sum(axis=0)))
final = pd.concat([acc_id,df2['Severity']],axis=1)
print(final)
print(df2['Severity'].unique())
final.to_csv('PlaneSubmission.csv',header=True , index=False)

[[2.6554648e-13 1.3003016e-16 1.7583617e-11 6.0000000e+00]
 [3.3042069e-11 6.0000000e+00 3.8254761e-08 3.1015460e-14]
 [3.7218240e-05 1.5694883e-10 5.9999628e+00 1.1914175e-08]
 ...
 [1.2360772e-06 2.9257894e-14 5.9999952e+00 3.2117302e-06]
 [1.9937593e-08 6.0284587e-19 6.0000000e+00 1.0002030e-07]
 [4.2778864e-29 4.4385272e-14 1.5818867e-11 6.0000000e+00]]
      Accident_ID                                 Severity
0               1                Highly_Fatal_And_Damaging
1              10        Significant_Damage_And_Fatalities
2              14  Significant_Damage_And_Serious_Injuries
3              17                Highly_Fatal_And_Damaging
4              21        Significant_Damage_And_Fatalities
5              40        Significant_Damage_And_Fatalities
6              45        Significant_Damage_And_Fatalities
7              50                Highly_Fatal_And_Damaging
8              52                Highly_Fatal_And_Damaging
9              85  Significant_Damage_And_Serious_